In [1]:
import numpy as np
import pandas as pd
import unidecode
import os

In [2]:
path = r'C:\Users\Franco\Dropbox\files\clean'

In [171]:
mylist = []

for file in os.listdir(r'C:\Users\Franco\Dropbox\files\properati'):
    if file.endswith(".csv"):
        mylist += [file]

In [172]:
prope = pd.DataFrame()

for file in mylist:
    df = pd.read_csv(r'C:\Users\Franco\Dropbox\files\properati\{}'.format(file))
    prope = prope.append(df)

In [173]:
prope = prope.drop(columns=['ad_type','rooms','bedrooms','bathrooms','title','description','lat','lon'])
prope = prope[prope['l1'].isin(['Argentina', 'Uruguay','Perú', 'Colombia','Ecuador'])]
prope = prope.loc[prope['operation_type']=='Venta',:]
prope = prope[prope['property_type'].isin(['Casa','Departamento','PH','Apartamento'])]

In [174]:
[prope['Country'],prope['City']] = [prope['l1'],prope['l3']]

prope = prope[prope['price'].notna()]
prope = prope[prope['City'].notna()]

Cambio de Buenos Aires y Montevideo

In [175]:
prope = prope.set_index(['l2'])

In [176]:
prope.at['Capital Federal','City'] = 'Ciudad de Buenos Aires'

In [177]:
prope.at['Montevideo','City'] = 'Montevideo'

In [178]:
prope = prope.reset_index()

In [179]:
prope = prope.replace({'Country':{'Perú':'Peru'}})

CAMBIAR NOBMRES

In [201]:
#Prope precios de prop's comerciales para obtener nombres limpios
p = pd.read_excel(r'C:\Users\Franco\Dropbox\files\clean\prope\comercial\properati_i.xlsx',index_col=[0,1])

In [202]:
p = p.loc[:,'City Original'].reset_index().drop_duplicates().rename(columns={'City':'City cw','City Original':'City'}).set_index(['Country','City'])

In [203]:
p = p.reset_index().append(pd.DataFrame([['Argentina','San Miguel de Tucumán','Gran Tucuman-Tafi Viejo']],columns=['Country','City','City cw'])).set_index(['Country','City'])

In [209]:
prope['City'] = prope['City'].apply(lambda x: unidecode.unidecode(x))

In [210]:
prope['Country'] = prope['Country'].apply(lambda x: unidecode.unidecode(x))

In [211]:
prope = prope.set_index(['Country','City'])

In [212]:
prope = prope.join(p,how='inner')

In [213]:
prope['fecha'] = prope['created_on'].str.strip()

In [214]:
prope = prope[prope['currency'].isin(['USD','COP'])]

In [215]:
prope = prope.loc[:,['surface_total','surface_covered','price','currency','property_type','City cw','fecha']]

Calcular metros promedio por pais y tipo de propiedad

In [219]:
inds = prope[['surface_total','surface_covered']].isnull().all(axis=1)

In [220]:
me = prope.loc[~inds,:]

In [237]:
def r(row):
    if row['property_type'] in ['Departamento','Apartamento']:
        val = 'Departamento'
    else:
        val = 'Casa'
    return val

In [221]:
me['Tipo Propiedad'] = me.apply(r,axis=1)

C:\ProgramData\Anaconda3\envs\caf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [222]:
me = me.groupby(['Country','Tipo Propiedad'])['surface_total','surface_covered'].mean()

C:\ProgramData\Anaconda3\envs\caf\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [225]:
me.to_csv(r'C:/Users/Franco/Dropbox\files/clean/prope/vivienda/metros_promedio.csv')

Currency

In [36]:
serie = os.listdir(r'c:/Users/Franco/Dropbox/files/properati')[0]

In [37]:
tc = pd.read_excel(r'c:/Users/Franco/Dropbox/files/properati/{}'.format(serie),usecols=[0,1],skiprows=[0,1,2,3,4,5,6]).dropna()

In [38]:
tc['fecha'] = tc[tc.columns[0]].astype(str).str.split(' ').str[0].str.strip()

In [39]:
tc = tc.rename(columns={'Tasa de cambio representativa del mercado (TRM)':'TDC'})

Join

In [226]:
prope = prope.drop(columns=['surface_total','surface_covered']) 

In [227]:
prope['price'] = prope['price'].astype('int')

Parto con las df's de prope porq es demasiado grande

In [228]:
prope = prope.reset_index().set_index('Country')

In [229]:
prope_no = prope.drop('Colombia')

In [230]:
prope_col = prope.reset_index()[prope.reset_index()['Country']=='Colombia']

In [231]:
prope_col = prope_col.merge(tc.loc[:,['fecha','TDC']],on='fecha',how='left',indicator=True,validate='m:1')

In [232]:
prope_no['TDC'] = np.nan

C:\ProgramData\Anaconda3\envs\caf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [233]:
prope_no = prope_no.reset_index()

In [234]:
prope = prope_no.append(prope_col.drop(columns='_merge'))

In [235]:
def f(row):
    if row['currency'] == 'USD':
        val = row['price']
    else:
        val = row['price'] / row['TDC']
    return val

In [236]:
prope['Precio USD'] = prope.apply(f,axis=1)

In [237]:
def r(row):
    if row['property_type'] in ['Departamento','Apartamento']:
        val = 'Departamento'
    else:
        val = 'Casa'
    return val

In [238]:
prope['Tipo Propiedad'] = prope.apply(r,axis=1)

In [239]:
prope = prope.rename(columns={'City':'City Original','City cw':'City'}).set_index(['Country','City'])

In [240]:
prope_vivienda = prope.drop(columns=['price','TDC','property_type']).groupby(['Country','City','Tipo Propiedad']).mean()

In [244]:
prope.to_csv(r'C:/Users/Franco/Dropbox\files/clean/prope/vivienda/prope_vivienda.csv')

In [245]:
prope_vivienda.to_csv(r'C:/Users/Franco/Dropbox\files/clean/prope/vivienda/prope_vivienda_promedios.csv')

### Ver tema cantidad de salarios para comprar vivienda

In [246]:
cw = pd.read_excel(r'C:/Users/Franco/Dropbox/files/clean/cwpops.xlsx',index_col=[0,1])

In [247]:
laboral = pd.read_excel(r'C:/Users/Franco/Dropbox/files/clean/laboral_2.xlsx',index_col=[0,1])

In [248]:
salarios = cw.loc[:,'pops'].to_frame().join(laboral.loc[:,'Salario Horario (PPP 2011)'],how='inner')

In [249]:
salarios['Salario Mensual'] = salarios['Salario Horario (PPP 2011)'] * 175

In [250]:
prope_vivienda = prope_vivienda.unstack()

In [259]:
salarios = salarios.join(prope_vivienda,how='inner').rename(columns={('Precio USD', 'Casa'):'Precio Promedio Casa (USD)',('Precio USD', 'Departamento'):'Precio Promedio Departamento (USD)'})

In [261]:
[salarios['Cantidad de salarios mensuales para adquirir Casa'],salarios['Cantidad de salarios mensuales para adquirir Departamento']] = [salarios['Precio Promedio Casa (USD)']/salarios['Salario Mensual'],salarios['Precio Promedio Departamento (USD)']/salarios['Salario Mensual']]

In [263]:
salarios.to_csv(r'C:/Users/Franco/Dropbox\files/clean/prope/vivienda/salariosxviviendo.csv')